In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import time
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.multiclass import type_of_target
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from scikeras.wrappers import KerasClassifier  # Use scikeras for scikit-learn compatibility
from math import floor
from bayes_opt import BayesianOptimization
from tensorflow.keras.layers import LeakyReLU  # Use tensorflow.keras instead of keras
LeakyReLU = LeakyReLU(negative_slope=0.1)
import warnings

# Google Colab upload utilities
from google.colab import files
import io

print("📤 Ready to upload data files into Colab!")
print("✅ All libraries loaded successfully.")


📤 Ready to upload data files into Colab!
✅ All libraries loaded successfully.


In [2]:
# Set option to ensure charts are displayed inline in the notebook

%matplotlib inline

In [3]:
# Set option to display max number of columns

pd.set_option('display.max_columns', None)

In [4]:
# --- STEP 1: Upload cleaned climate dataset ---
print("📤 Upload your cleaned climate dataset (e.g., climate_clean_data_new.csv)")
uploaded = files.upload()

# Load the first uploaded file
first_file = list(uploaded.keys())[0]
climate = pd.read_csv(io.BytesIO(uploaded[first_file]))
print(f"✅ Loaded cleaned climate dataset: {first_file}")

# Preview
print("\n🌦 Climate data preview:")
display(climate.head())


# --- STEP 2: Upload prediction dataset ---
print("\n📤 Upload 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'")
uploaded = files.upload()

# Load the first uploaded file
first_file = list(uploaded.keys())[0]
prediction = pd.read_csv(io.BytesIO(uploaded[first_file]))
print(f"✅ Loaded prediction dataset: {first_file}")

# Preview
print("\n🔎 Prediction data preview:")
display(prediction.head())


📤 Upload your cleaned climate dataset (e.g., climate_clean_data_new.csv)


Saving climate_clean_data.csv to climate_clean_data.csv
✅ Loaded cleaned climate dataset: climate_clean_data.csv

🌦 Climate data preview:


,BASEL_cloud_cover,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_sunshine,BASEL_temp_mean,BASEL_temp_min,BASEL_temp_max,BELGRADE_cloud_cover,BELGRADE_humidity,BELGRADE_pressure,BELGRADE_global_radiation,BELGRADE_precipitation,BELGRADE_sunshine,BELGRADE_temp_mean,BELGRADE_temp_min,BELGRADE_temp_max,BUDAPEST_cloud_cover,BUDAPEST_humidity,BUDAPEST_pressure,BUDAPEST_global_radiation,BUDAPEST_precipitation,BUDAPEST_sunshine,BUDAPEST_temp_mean,BUDAPEST_temp_min,BUDAPEST_temp_max,DEBILT_cloud_cover,DEBILT_humidity,DEBILT_pressure,DEBILT_global_radiation,DEBILT_precipitation,DEBILT_sunshine,DEBILT_temp_mean,DEBILT_temp_min,DEBILT_temp_max,DUSSELDORF_cloud_cover,DUSSELDORF_humidity,DUSSELDORF_pressure,DUSSELDORF_global_radiation,DUSSELDORF_precipitation,DUSSELDORF_sunshine,DUSSELDORF_temp_mean,DUSSELDORF_temp_min,DUSSELDORF_temp_max,HEATHROW_cloud_cover,HEATHROW_humidity,HEATHROW_pressure,HEATHROW_global_radiation,HEATHROW_precipitation,HEATHROW_sunshine,HEATHROW_temp_mean,HEATHROW_temp_min,HEATHROW_temp_max,KASSEL_cloud_cover,KASSEL_humidity,KASSEL_pressure,KASSEL_global_radiation,KASSEL_precipitation,KASSEL_sunshine,KASSEL_temp_mean,KASSEL_temp_min,KASSEL_temp_max,LJUBLJANA_cloud_cover,LJUBLJANA_humidity,LJUBLJANA_pressure,LJUBLJANA_global_radiation,LJUBLJANA_precipitation,LJUBLJANA_sunshine,LJUBLJANA_temp_mean,LJUBLJANA_temp_min,LJUBLJANA_temp_max,MAASTRICHT_cloud_cover,MAASTRICHT_humidity,MAASTRICHT_pressure,MAASTRICHT_global_radiation,MAASTRICHT_precipitation,MAASTRICHT_sunshine,MAASTRICHT_temp_mean,MAASTRICHT_temp_min,MAASTRICHT_temp_max,MADRID_cloud_cover,MADRID_humidity,MADRID_pressure,MADRID_global_radiation,MADRID_precipitation,MADRID_sunshine,MADRID_temp_mean,MADRID_temp_min,MADRID_temp_max,MUNCHENB_cloud_cover,MUNCHENB_humidity,MUNCHENB_pressure,MUNCHENB_global_radiation,MUNCHENB_precipitation,MUNCHENB_sunshine,MUNCHENB_temp_mean,MUNCHENB_temp_min,MUNCHENB_temp_max,OSLO_cloud_cover,OSLO_humidity,OSLO_pressure,OSLO_global_radiation,OSLO_precipitation,OSLO_sunshine,OSLO_temp_mean,OSLO_temp_min,OSLO_temp_max,SONNBLICK_cloud_cover,SONNBLICK_humidity,SONNBLICK_pressure,SONNBLICK_global_radiation,SONNBLICK_precipitation,SONNBLICK_sunshine,SONNBLICK_temp_mean,SONNBLICK_temp_min,SONNBLICK_temp_max,STOCKHOLM_cloud_cover,STOCKHOLM_humidity,STOCKHOLM_pressure,STOCKHOLM_global_radiation,STOCKHOLM_precipitation,STOCKHOLM_sunshine,STOCKHOLM_temp_mean,STOCKHOLM_temp_min,STOCKHOLM_temp_max,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,7,0.85,1.018,0.32,0.09,0.7,6.5,0.8,10.9,1,0.81,1.0195,0.88,0.00,7.0,3.7,-0.9,7.9,4,0.67,1.017,0.44,0.01,2.3,2.4,-0.4,5.1,7,0.85,1.0032,0.07,0.25,0.0,9.3,7.4,11.0,8,0.83,1.0161,0.12,0.08,0.0,10.0,7.0,11.5,7,0.91,1.0010,0.13,0.22,0.0,10.6,9.4,8.3,8,0.82,1.0094,0.28,0.48,1.6,7.9,3.9,9.4,8,1.00,1.0173,0.20,0.00,0.0,-0.6,-1.9,0.5,7,0.83,1.0063,0.22,0.32,1.0,9.5,8.5,11.1,6,0.92,1.0260,0.53,0.0,1.4,7.6,4.4,10.8,5,0.67,1.018,0.20,0.10,0.0,6.9,1.1,10.4,8,0.98,0.9978,0.04,1.14,0.0,4.9,3.8,5.9,4,0.73,1.0304,0.48,0.01,2.3,-5.9,-8.5,-3.2,5,0.98,1.0114,0.05,0.32,0.0,4.2,2.2,4.9,5,0.88,1.0003,0.45,0.34,4.7,8.5,6.0,10.9
1,6,0.84,1.018,0.36,1.05,1.1,6.1,3.3,10.1,6,0.84,1.0172,0.25,0.00,0.0,2.9,2.2,4.4,4,0.67,1.017,0.18,0.31,0.0,2.3,1.4,3.1,8,0.90,1.0056,0.14,0.06,0.1,7.7,6.4,8.3,8,0.89,1.0161,0.18,0.66,0.5,8.2,7.4,11.0,7,0.98,1.0051,0.13,0.23,0.0,6.1,3.9,10.6,8,0.86,1.0086,0.12,0.27,0.0,7.7,6.8,9.1,6,0.94,1.0173,0.56,0.13,3.2,2.1,-1.3,5.5,8,0.92,1.0062,0.17,1.34,0.4,8.6,7.5,9.9,7,0.86,1.0254,0.46,0.0,0.9,9.8,7.4,12.2,6,0.72,1.018,0.61,0.30,5.1,6.2,4.2,10.2,8,0.62,1.0139,0.04,0.00,0.0,3.4,2.8,4.9,6,0.97,1.0292,0.21,0.61,0.0,-9.5,-10.5,-8.5,5,0.62,1.0114,0.05,0.06,0.0,4.0,3.0,5.0,7,0.91,1.0007,0.25,0.84,0.7,8.9,5.6,12.1
2,8,0.90,1.018,0.18,0.30,0.0,8.5,5.1,9.9,6,0.77,1.0179,0.67,0.00,3.5,3.1,-0.5,6.4,4,0.67,1.017,0.30,0.00,0.6,2.7,1.7,5.3,6,0.92,1.0165,0.28,0.01,


📤 Upload 'Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'


Saving Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv to Dataset-Answers-Weather_Prediction_Pleasant_Weather (2).csv
✅ Loaded prediction dataset: Dataset-Answers-Weather_Prediction_Pleasant_Weather (2).csv

🔎 Prediction data preview:


,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# Drop DATE column from answers

prediction.drop(columns = 'DATE', inplace = True)

In [6]:
climate.shape

(22950, 135)

In [9]:
climate.shape

(22950, 135)

In [10]:
# Turn X and answers from a df to arrays

X = np.array(climate)
y = np.array(prediction)

In [11]:
X = X.reshape(-1,15,9)

In [12]:
X.shape

(22950, 15, 9)

In [13]:
# Use argmax to transform y

y =  np.argmax(y, axis = 1)
y

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
y.shape

(22950,)

In [15]:
# Check y layout

from sklearn.utils.multiclass import type_of_target
type_of_target(y)

'multiclass'

In [16]:
# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [17]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(17212, 15, 9) (17212,)
(5738, 15, 9) (5738,)


In [18]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15 # Number of weather stations
# Make scorer accuracy
score_acc = make_scorer(accuracy_score)

In [19]:
# Create function

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    #optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate),
                 #'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate),
                 #'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate),
                 #'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU,'relu']

    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]  #optimizerD[optimizerL[round(optimizer)]]
    optimizer = optimizerL[round(optimizer)]
    batch_size = round(batch_size)

    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)

    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel,activation=activation, input_shape=(timesteps, input_dim)))
        #model.add(Conv1D(32, kernel_size=1,activation='relu', input_shape=(timesteps, input_dim)))

        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation)) #(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax')) #sigmoid softmax
        #model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) #categorical_crossentropy
        return model
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_acc, cv=kfold, params={'callbacks':[es]}).mean()
    return score

In [21]:
# To fix the AttributeError: 'super' object has no attribute '__sklearn_tags__',
# we need to ensure that the scikeras library is compatible with the scikit-learn version.
# The most common solution is to upgrade scikeras to its latest version.
# Let's first install the latest scikeras version.

# !pip install --upgrade scikeras # Already run, and scikeras is up to date

# The error persists, indicating a potential incompatibility with the very latest scikit-learn.
# Let's downgrade scikit-learn to a known compatible version (e.g., 1.5.1).
!pip install scikit-learn==1.5.1

# After upgrading/downgrading, you MUST restart the runtime (Runtime -> Restart runtime)
# and then re-run all cells, or at least re-import KerasClassifier if it's already imported.

# Then, re-run the Bayesian Optimization block.

params = {
    'neurons': (20, 50),           # smaller range, fewer neurons -> faster
    'kernel': (1, 2),              # reduce kernel sizes for speed
    'activation': (0, 3),          # limit to common activations (relu, tanh, sigmoid)
    'optimizer': (0, 3),           # choose among faster optimizers (SGD, Adam, RMSprop)
    'learning_rate': (0.01, 0.1),  # smaller range, faster convergence
    'batch_size': (256, 512),      # mid-size batches are faster than tiny ones
    'epochs': (20, 40),            # fewer epochs to reduce runtime
    'layers1': (1, 2),             # fewer layers to speed up
    'layers2': (0, 1),             # optional second layer
    'normalization': (0, 1),       # keep as is (binary choice)
    'dropout': (0, 1),             # keep as is
    'dropout_rate': (0, 0.2)       # smaller dropout to avoid overkill
}

# Run Bayesian Optimization
nn_opt = BayesianOptimization(bay_area, params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4) #25
print('Search took %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |  neurons  |  kernel   | activa... | optimizer | learni... | batch_... |  epochs   |  layers1  |  layers2  | normal... |  dropout  | dropou... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/21


/usr/local/lib/python3.12/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 3s - 57ms/step - accuracy: 0.6037 - loss: 1.3456
Epoch 2/21
47/47 - 1s - 22ms/step - accuracy: 0.7019 - loss: 0.9164
Epoch 3/21
47/47 - 1s - 22ms/step - accuracy: 0.7149 - loss: 0.8528
Epoch 4/21
47/47 - 1s - 22ms/step - accuracy: 0.7299 - loss: 0.8038
Epoch 5/21
47/47 - 1s - 22ms/step - accuracy: 0.7363 - loss: 0.7792
Epoch 6/21
47/47 - 1s - 30ms/step - accuracy: 0.7423 - loss: 0.7543
Epoch 7/21
47/47 - 2s - 38ms/step - accuracy: 0.7502 - loss: 0.7277
Epoch 8/21
47/47 - 2s - 38ms/step - accuracy: 0.7536 - loss: 0.7122
Epoch 9/21
47/47 - 1s - 22ms/step - accuracy: 0.7566 - loss: 0.6921
Epoch 10/21
47/47 - 1s - 23ms/step - accuracy: 0.7637 - loss: 0.6746
Epoch 11/21
47/47 - 1s - 22ms/step - accuracy: 0.7698 - loss: 0.6519
Epoch 12/21
47/47 - 1s - 22ms/step - accuracy: 0.7711 - loss: 0.6404
Epoch 13/21
47/47 - 1s - 29ms/step - accuracy: 0.7784 - loss: 0.6230
Epoch 14/21
47/47 - 1s - 23ms/step - accuracy: 0.7810 - loss: 0.6083
Epoch 15/21
47/47 - 1s - 23ms/step - accuracy: 0.7831 

/usr/local/lib/python3.12/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 3s - 70ms/step - accuracy: 0.6357 - loss: 1.1329
Epoch 2/21
47/47 - 2s - 39ms/step - accuracy: 0.6775 - loss: 0.9405
Epoch 3/21
47/47 - 1s - 21ms/step - accuracy: 0.7104 - loss: 0.8450
Epoch 4/21
47/47 - 1s - 21ms/step - accuracy: 0.7333 - loss: 0.7824
Epoch 5/21
47/47 - 1s - 22ms/step - accuracy: 0.7453 - loss: 0.7426
Epoch 6/21
47/47 - 1s - 21ms/step - accuracy: 0.7523 - loss: 0.7105
Epoch 7/21
47/47 - 1s - 21ms/step - accuracy: 0.7630 - loss: 0.6844
Epoch 8/21
47/47 - 1s - 21ms/step - accuracy: 0.7680 - loss: 0.6616
Epoch 9/21
47/47 - 1s - 21ms/step - accuracy: 0.7778 - loss: 0.6360
Epoch 10/21
47/47 - 1s - 21ms/step - accuracy: 0.7796 - loss: 0.6178
Epoch 11/21
47/47 - 1s - 25ms/step - accuracy: 0.7881 - loss: 0.5997
Epoch 12/21
47/47 - 2s - 36ms/step - accuracy: 0.7908 - loss: 0.5850
Epoch 13/21
47/47 - 1s - 27ms/step - accuracy: 0.7972 - loss: 0.5678
Epoch 14/21
47/47 - 1s - 21ms/step - accuracy: 0.8033 - loss: 0.5511
Epoch 15/21
47/47 - 1s - 26ms/step - accuracy: 0.8029 

...

47/47 - 2s - 47ms/step - accuracy: 0.6419 - loss: 1.2999
Epoch 2/21
47/47 - 2s - 37ms/step - accuracy: 0.6803 - loss: 0.9210
Epoch 3/21
47/47 - 2s - 43ms/step - accuracy: 0.7040 - loss: 0.8425
Epoch 4/21
47/47 - 1s - 27ms/step - accuracy: 0.7225 - loss: 0.7860
Epoch 5/21
47/47 - 1s - 16ms/step - accuracy: 0.7373 - loss: 0.7394
Epoch 6/21
47/47 - 1s - 16ms/step - accuracy: 0.7553 - loss: 0.7001
Epoch 7/21
47/47 - 1s - 17ms/step - accuracy: 0.7656 - loss: 0.6683
Epoch 8/21
47/47 - 1s - 17ms/step - accuracy: 0.7744 - loss: 0.6466
Epoch 9/21
47/47 - 1s - 16ms/step - accuracy: 0.7821 - loss: 0.6216
Epoch 10/21
47/47 - 1s - 27ms/step - accuracy: 0.7852 - loss: 0.6023
Epoch 11/21
47/47 - 1s - 16ms/step - accuracy: 0.7919 - loss: 0.5838
Epoch 12/21
47/47 - 1s - 17ms/step - accuracy: 0.7968 - loss: 0.5685
Epoch 13/21
47/47 - 1s - 17ms/step - accuracy: 0.8027 - loss: 0.5527
Epoch 14/21
47/47 - 1s - 26ms/step - accuracy: 0.8123 - loss: 0.5337
Epoch 15/21
47/47 - 1s - 28ms/step - accuracy: 0.8143 

/usr/local/lib/python3.12/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 2s - 47ms/step - accuracy: 0.6448 - loss: 1.2766
Epoch 2/21
47/47 - 1s - 18ms/step - accuracy: 0.7016 - loss: 0.8842
Epoch 3/21
47/47 - 1s - 26ms/step - accuracy: 0.7323 - loss: 0.7888
Epoch 4/21
47/47 - 1s - 21ms/step - accuracy: 0.7495 - loss: 0.7321
Epoch 5/21
47/47 - 1s - 28ms/step - accuracy: 0.7686 - loss: 0.6852
Epoch 6/21
47/47 - 1s - 28ms/step - accuracy: 0.7814 - loss: 0.6477
Epoch 7/21
47/47 - 2s - 44ms/step - accuracy: 0.7885 - loss: 0.6180
Epoch 8/21
47/47 - 1s - 17ms/step - accuracy: 0.8002 - loss: 0.5925
Epoch 9/21
47/47 - 1s - 27ms/step - accuracy: 0.8057 - loss: 0.5732
Epoch 10/21
47/47 - 1s - 27ms/step - accuracy: 0.8099 - loss: 0.5554
Epoch 11/21
47/47 - 1s - 27ms/step - accuracy: 0.8145 - loss: 0.5388
Epoch 12/21
47/47 - 1s - 26ms/step - accuracy: 0.8203 - loss: 0.5247
Epoch 13/21
47/47 - 1s - 27ms/step - accuracy: 0.8240 - loss: 0.5076
Epoch 14/21
47/47 - 1s - 18ms/step - accuracy: 0.8280 - loss: 0.4944
Epoch 15/21
47/47 - 1s - 29ms/step - accuracy: 0.8348 

/usr/local/lib/python3.12/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


47/47 - 2s - 48ms/step - accuracy: 0.6309 - loss: 1.3094
Epoch 2/21
47/47 - 1s - 17ms/step - accuracy: 0.6783 - loss: 0.9241
Epoch 3/21
47/47 - 1s - 25ms/step - accuracy: 0.7137 - loss: 0.8272
Epoch 4/21
47/47 - 1s - 30ms/step - accuracy: 0.7372 - loss: 0.7644
Epoch 5/21
47/47 - 1s - 23ms/step - accuracy: 0.7498 - loss: 0.7192
Epoch 6/21
47/47 - 1s - 17ms/step - accuracy: 0.7643 - loss: 0.6834
Epoch 7/21
47/47 - 1s - 17ms/step - accuracy: 0.7731 - loss: 0.6544
Epoch 8/21
47/47 - 1s - 17ms/step - accuracy: 0.7815 - loss: 0.6317
Epoch 9/21
47/47 - 1s - 26ms/step - accuracy: 0.7869 - loss: 0.6103
Epoch 10/21
47/47 - 1s - 27ms/step - accuracy: 0.7972 - loss: 0.5936
Epoch 11/21
47/47 - 1s - 16ms/step - accuracy: 0.8006 - loss: 0.5748
Epoch 12/21
47/47 - 1s - 17ms/step - accuracy: 0.8049 - loss: 0.5630
Epoch 13/21
47/47 - 1s - 17ms/step - accuracy: 0.8088 - loss: 0.5512
Epoch 14/21
47/47 - 1s - 17ms/step - accuracy: 0.8134 - loss: 0.5376
Epoch 15/21
47/47 - 1s - 17ms/step - accuracy: 0.8171 

In [22]:
optimum = nn_opt.max['params']
learning_rate = optimum['learning_rate']

activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', LeakyReLU, 'relu']
optimum['activation'] = activationL[round(optimum['activation'])]

optimum['batch_size'] = round(optimum['batch_size'])
optimum['epochs'] = round(optimum['epochs'])
optimum['layers1'] = round(optimum['layers1'])
optimum['layers2'] = round(optimum['layers2'])
optimum['neurons'] = round(optimum['neurons'])

optimizerL = ['Adam', 'SGD', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'Adam']
optimizerD = {
    'Adam': Adam(learning_rate=learning_rate),
    'SGD': SGD(learning_rate=learning_rate),
    'RMSprop': RMSprop(learning_rate=learning_rate),
    'Adadelta': Adadelta(learning_rate=learning_rate),
    'Adagrad': Adagrad(learning_rate=learning_rate),
    'Adamax': Adamax(learning_rate=learning_rate),
    'Nadam': Nadam(learning_rate=learning_rate),
    'Ftrl': Ftrl(learning_rate=learning_rate)
}
optimum['optimizer'] = optimizerD[optimizerL[round(optimum['optimizer'])]]
optimum

{'neurons': 34,
 'kernel': np.float64(2.0),
 'activation': 'softsign',
 'optimizer': <keras.src.optimizers.rmsprop.RMSprop at 0x7fc740934f20>,
 'learning_rate': np.float64(0.01),
 'batch_size': 295,
 'epochs': 21,
 'layers1': 2,
 'layers2': 1,
 'normalization': np.float64(0.8367439429343153),
 'dropout': np.float64(0.0),
 'dropout_rate': np.float64(0.2)}

In [23]:
# Set the model with optimized hyperparameters

epochs = 21
batch_size = 295

timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = 15

layers1 = 2
layers2 = 1
activation = 'softsign'
kernel = int(round(2.0))  # Rounded kernel size for Conv1D
neurons = 34
normalization = 0.8367439429343153
dropout = 0.0
dropout_rate = 0.2
optimizer = RMSprop(learning_rate=0.01)  # Instantiate RMSprop with provided learning rate

model = Sequential()
model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))

if normalization > 0.5:
    model.add(BatchNormalization())

for i in range(layers1):
    model.add(Dense(neurons, activation=activation))

if dropout > 0.5:
    model.add(Dropout(dropout_rate))

for i in range(layers2):
    model.add(Dense(neurons, activation=activation))

model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [24]:
model.summary()

Model: "sequential_101"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_101 (Conv1D)             │ (None, 14, 34)         │           646 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_60          │ (None, 14, 34)         │           136 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_355 (Dense)               │ (None, 14, 34)         │         1,190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_356 (Dense)               │ (None, 14, 34)         │         1,190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_357 (Dense)               │ (None, 14, 34)         │         1,190 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_101               │ (None, 7, 34)          │             0 │
│ (MaxPooling1D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_101 (Flatten)           │ (None, 238)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_358 (Dense)               │ (None, 15)             │         3,585 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 7,937 (31.00 KB)

 Trainable params: 7,869 (30.74 KB)

 Non-trainable params: 68 (272.00 B)

In [25]:
# Put the y_test set back into a one-hot configuration

y_train_one_hot = to_categorical(y_train, num_classes=15)

In [26]:
# Check shapes

print(f'X_train shape: {X_train.shape}')
print(f'y_train_one_hot shape: {y_train_one_hot.shape}')

X_train shape: (17212, 15, 9)
y_train_one_hot shape: (17212, 15)


In [27]:
# Compile the model with categorical_crossentropy

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [28]:
# Fit the model to the data

model.fit(X_train, y_train_one_hot, batch_size=batch_size, epochs=epochs, verbose=2)

Epoch 1/21
59/59 - 4s - 68ms/step - accuracy: 0.6855 - loss: 0.9282
Epoch 2/21
59/59 - 1s - 17ms/step - accuracy: 0.7616 - loss: 0.6804
Epoch 3/21
59/59 - 1s - 17ms/step - accuracy: 0.7847 - loss: 0.6019
Epoch 4/21
59/59 - 1s - 16ms/step - accuracy: 0.7974 - loss: 0.5697
Epoch 5/21
59/59 - 1s - 16ms/step - accuracy: 0.8141 - loss: 0.5240
Epoch 6/21
59/59 - 1s - 16ms/step - accuracy: 0.8240 - loss: 0.4909
Epoch 7/21
59/59 - 1s - 22ms/step - accuracy: 0.8334 - loss: 0.4718
Epoch 8/21
59/59 - 1s - 21ms/step - accuracy: 0.8399 - loss: 0.4501
Epoch 9/21
59/59 - 1s - 23ms/step - accuracy: 0.8453 - loss: 0.4337
Epoch 10/21
59/59 - 3s - 49ms/step - accuracy: 0.8496 - loss: 0.4201
Epoch 11/21
59/59 - 3s - 46ms/step - accuracy: 0.8704 - loss: 0.3727
Epoch 12/21
59/59 - 2s - 35ms/step - accuracy: 0.8700 - loss: 0.3701
Epoch 13/21
59/59 - 1s - 25ms/step - accuracy: 0.8716 - loss: 0.3647
Epoch 14/21
59/59 - 1s - 17ms/step - accuracy: 0.8800 - loss: 0.3461
Epoch 15/21
59/59 - 1s - 17ms/step - accura

In [29]:
# Define list of stations names

stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'
}

In [30]:
def confusion_matrix(y_true, y_pred, stations):
    # Check if y_true and y_pred are one-hot encoded or already class indices
    if y_true.ndim == 1:
        y_true_labels = y_true
    else:
        y_true_labels = np.argmax(y_true, axis=1)

    if y_pred.ndim == 1:
        y_pred_labels = y_pred
    else:
        y_pred_labels = np.argmax(y_pred, axis=1)

    # Map numeric labels to activity names
    y_true_series = pd.Series([stations[y] for y in y_true_labels])
    y_pred_series = pd.Series([stations[y] for y in y_pred_labels])

    return pd.crosstab(y_true_series, y_pred_series, rownames=['True'], colnames=['Pred'])

In [31]:
y_pred = model.predict(X_test)

180/180 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [32]:
# Evaluate

print(confusion_matrix(y_test, y_pred, stations))

Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL        3555        73         7       5           3         2       0   
BELGRADE      316       772         0       2           0         0       0   
BUDAPEST       69        57        86       1           0         0       0   
DEBILT         33        10        10      29           0         0       0   
DUSSELDORF     10         4         4       6           4         1       0   
HEATHROW       20         9         6       1           8        34       0   
KASSEL          2         2         1       0           2         0       3   
LJUBLJANA      18         2         3       0           0         0       1   
MAASTRICHT      8         0         0       0           0         0       0   
MADRID         39        31        23       3           5        12       0   
MUNCHENB        7         1         0       0       